In [24]:
import sqlite3
import pandas as pd

In [25]:
con = sqlite3.connect(':memory:')

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [26]:
data = pd.read_csv('boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [27]:
data.to_sql('boston', con, index=False, if_exists='replace')

506

In [28]:
tasks = 0

### Все задания нужно выполнить только с помощью sql

1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [29]:
tasks+=1
pd.read_sql('''

   SELECT
   (SELECT COUNT(CRIM) FROM boston WHERE CRIM IS NULL) AS CRIM,
   (SELECT COUNT(ZN) FROM boston where ZN IS NULL) AS ZN,
   (SELECT COUNT(INDUS) FROM boston where INDUS IS NULL) AS INDUS,
   (SELECT COUNT(CHAS) FROM boston where CHAS IS NULL) AS CHAS,
   (SELECT COUNT(NOX) FROM boston where NOX IS NULL) AS NOX

''', con,)
# В таблице нет значений NULL


,CRIM,ZN,INDUS,CHAS,NOX
0,0,0,0,0,0


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [30]:
tasks+=1
pd.read_sql(
    '''
   SELECT
   (SELECT COUNT(DISTINCT CRIM) FROM boston) AS CRIM,
    (SELECT COUNT(DISTINCT ZN) FROM boston) AS ZN,
    (SELECT COUNT(DISTINCT INDUS) FROM boston) AS INDUS,
    (SELECT COUNT(DISTINCT CHAS) FROM boston ) AS CHAS,
    (SELECT COUNT(DISTINCT NOX)  FROM boston ) AS NOX
    ''',
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX
0,504,26,76,2,81


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [38]:
tasks+=1
pd.read_sql(
    '''
WITH MedianCRIM AS (
    SELECT AVG(CRIM) AS MedianValue
    FROM (
        SELECT CRIM
        FROM BOSTON
        ORDER BY CRIM
        LIMIT 2 - (SELECT COUNT(*) FROM BOSTON) % 2
        OFFSET (SELECT (COUNT(*) - 1) / 2 FROM BOSTON))),
  MinCRIM AS (
      SELECT MIN(CRIM) AS MinValue
      FROM BOSTON
  ),
  MedianZN AS (
      SELECT AVG(ZN) AS MedianValue
      FROM (
          SELECT ZN
          FROM BOSTON
          ORDER BY ZN
          LIMIT 2 - (SELECT COUNT(*) FROM BOSTON) % 2
          OFFSET (SELECT (COUNT(*) - 1) / 2 FROM BOSTON)
      )
  ),
  MinZN AS (
      SELECT MIN(ZN) AS MinValue
      FROM BOSTON
  ),
  MedianINDUS AS (
      SELECT AVG(INDUS) AS MedianValue
      FROM (
          SELECT INDUS
          FROM BOSTON
          ORDER BY INDUS
          LIMIT 2 - (SELECT COUNT(*) FROM BOSTON) % 2
          OFFSET (SELECT (COUNT(*) - 1) / 2 FROM BOSTON)
      )
  ),
  MinINDUS AS (
      SELECT MIN(INDUS) AS MinValue
      FROM BOSTON
  ),
  MedianCHAS AS (
      SELECT AVG(CHAS) AS MedianValue
      FROM (
          SELECT CHAS
          FROM BOSTON
          ORDER BY CHAS
          LIMIT 2 - (SELECT COUNT(*) FROM BOSTON) % 2
          OFFSET (SELECT (COUNT(*) - 1) / 2 FROM BOSTON)
      )
  ),
  MinCHAS AS (
      SELECT MIN(CHAS) AS MinValue
      FROM BOSTON
  ),
  MedianNOX AS (
      SELECT AVG(NOX) AS MedianValue
      FROM (
          SELECT NOX
          FROM BOSTON
          ORDER BY NOX
          LIMIT 2 - (SELECT COUNT(*) FROM BOSTON) % 2
          OFFSET (SELECT (COUNT(*) - 1) / 2 FROM BOSTON)
      )
  ),
  MinNOX AS (
      SELECT MIN(NOX) AS MinValue
      FROM BOSTON
  )

  SELECT
      'CRIM' AS ColumnName, MedianCRIM.MedianValue
      FROM MedianCRIM, MinCRIM WHERE MedianCRIM.MedianValue = MinCRIM.MinValue
  UNION ALL
  SELECT
      'ZN' AS ColumnName, MedianZN.MedianValue
      FROM MedianZN, MinZN WHERE MedianZN.MedianValue = MinZN.MinValue
  UNION ALL
  SELECT
      'INDUS' AS ColumnName, MedianINDUS.MedianValue
      FROM MedianINDUS, MinINDUS WHERE MedianINDUS.MedianValue = MinINDUS.MinValue
  UNION ALL
  SELECT
      'CHAS' AS ColumnName, MedianCHAS.MedianValue
      FROM MedianCHAS, MinCHAS WHERE MedianCHAS.MedianValue = MinCHAS.MinValue
  UNION ALL
  SELECT
      'NOX' AS ColumnName, MedianNOX.MedianValue
      FROM MedianNOX, MinNOX WHERE MedianNOX.MedianValue = MinNOX.MinValue;
   ''',
    con,
)
#В колонках ZN и CHAS медиана равна минимальному значению, это указывает на то, что половина набора данных состоит из значений, равных минимальному значению, или что распределение данных сильно смещено в сторону минимального значения.

,ColumnName,MedianValue
0,ZN,0.0
1,CHAS,0.0


4. Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами.
Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница).
Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [32]:
tasks+=1
pd.read_sql(
    '''
    SELECT
  '25' AS number_of_houses,
  (SELECT AVG(RM) FROM (SELECT RM FROM boston ORDER BY MEDV ASC LIMIT 25)) AS AVG_RM_low_priced,
  (SELECT AVG(RM) FROM (SELECT RM FROM boston ORDER BY MEDV DESC LIMIT 25)) AS AVG_RM_high_priced,
  (SELECT AVG(RM) FROM (SELECT RM FROM boston ORDER BY MEDV DESC LIMIT 25)) -
  (SELECT AVG(RM) FROM (SELECT RM FROM boston ORDER BY MEDV ASC LIMIT 25)) AS difference
UNION ALL
SELECT
  '50',
  (SELECT AVG(RM) FROM (SELECT RM FROM boston ORDER BY MEDV ASC LIMIT 50)),
  (SELECT AVG(RM) FROM (SELECT RM FROM boston ORDER BY MEDV DESC LIMIT 50)),
  (SELECT AVG(RM) FROM (SELECT RM FROM boston ORDER BY MEDV DESC LIMIT 50)) -
  (SELECT AVG(RM) FROM (SELECT RM FROM boston ORDER BY MEDV ASC LIMIT 50))
UNION ALL
SELECT
  '100',
  (SELECT AVG(RM) FROM (SELECT RM FROM boston ORDER BY MEDV ASC LIMIT 100)),
  (SELECT AVG(RM) FROM (SELECT RM FROM boston ORDER BY MEDV DESC LIMIT 100)),
  (SELECT AVG(RM) FROM (SELECT RM FROM boston ORDER BY MEDV DESC LIMIT 100)) -
  (SELECT AVG(RM) FROM (SELECT RM FROM boston ORDER BY MEDV ASC LIMIT 100))
UNION ALL
SELECT
  '200',
  (SELECT AVG(RM) FROM (SELECT RM FROM boston ORDER BY MEDV ASC LIMIT 200)),
  (SELECT AVG(RM) FROM (SELECT RM FROM boston ORDER BY MEDV DESC LIMIT 200)),
  (SELECT AVG(RM) FROM (SELECT RM FROM boston ORDER BY MEDV DESC LIMIT 200)) -
  (SELECT AVG(RM) FROM (SELECT RM FROM boston ORDER BY MEDV ASC LIMIT 200))
UNION ALL
SELECT
  '300',
  (SELECT AVG(RM) FROM (SELECT RM FROM boston ORDER BY MEDV ASC LIMIT 300)),
  (SELECT AVG(RM) FROM (SELECT RM FROM boston ORDER BY MEDV DESC LIMIT 300)),
  (SELECT AVG(RM) FROM (SELECT RM FROM boston ORDER BY MEDV DESC LIMIT 300)) -
  (SELECT AVG(RM) FROM (SELECT RM FROM boston ORDER BY MEDV ASC LIMIT 300));
    ''',
    con,
)
#Между самыми дорогими и самыми дешевыми домами разница не более 2 комнат, что означает высокую разницу в стоимости и небольшую разницу в количестве комнат.
#На стоимость это влияет не сильно, самые дорогие дома имеют комнат всего на 23% больше, чем самые дешевые

,number_of_houses,AVG_RM_low_priced,AVG_RM_high_priced,difference
0,25,5.747840,7.637320,1.889480
1,50,5.753240,7.487060,1.733820
2,100,5.887120,7.198760,1.311640
3,200,5.911705,6.758680,0.846975
4,300,5.972227,6.539007,0.566780


5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг) среди всех значений LSTAT. Напишите какой вывод можно сделать по этим данным.

In [33]:
tasks+=1
pd.read_sql(
    '''
   WITH RankedLSTAT AS (
  SELECT LSTAT, MEDV, RANK() OVER (ORDER BY LSTAT) AS Rank
  FROM boston
)
SELECT
  rl.LSTAT,
  rl.MEDV,
  rl.Rank
FROM
  RankedLSTAT rl
JOIN
  (SELECT MAX(MEDV) AS MaxMEDV FROM boston) AS MaxD
ON
  rl.MEDV = MaxD.MaxMEDV
ORDER BY
  rl.LSTAT;
    ''',
    con,
)
# Чем дороже дома, тем ниже процент более низкого по статусу населения.

,LSTAT,MEDV,Rank
0,1.73,50.0,1
1,1.92,50.0,2
2,2.88,50.0,6
3,2.96,50.0,8
4,2.97,50.0,9
5,3.16,50.0,15
6,3.26,50.0,17
7,3.32,50.0,18
8,3.70,50.0,26
9,3.73,50.0,27


6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [34]:
tasks+=1
pd.read_sql(
    '''
  SELECT
    (SELECT AVG(MEDV) FROM boston WHERE CHAS = 0) AS CHAS_0,
    (SELECT AVG(MEDV) FROM boston WHERE CHAS = 1) AS CHAS_1;




    ''',
    con,
)

,CHAS_0,CHAS_1
0,22.093843,28.44


7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [35]:
tasks+=1
pd.read_sql(
    '''
    WITH AVG_CRIM_CHAS_0 AS (
      SELECT AVG(CRIM) AS AVG_CRIM FROM boston
      WHERE CHAS = 0),

      AVG_CRIM_CHAS_1 AS(
        SELECT AVG(CRIM) AS AVG_CRIM1 FROM boston
      WHERE CHAS = 1
      ),
      AVG_ZN_CHAS_0 AS(
        SELECT AVG(ZN) AS AVG_ZN  FROM boston
      WHERE CHAS = 0
      ),
      AVG_ZN_CHAS_1 AS(
        SELECT AVG(ZN) AS AVG_ZN1 FROM boston
      WHERE CHAS = 1
      ),
      AVG_INDUS_CHAS_0 AS(
        SELECT AVG(INDUS) AS AVG_INDUS FROM boston
      WHERE CHAS = 0
      ),
       AVG_INDUS_CHAS_1 AS(
        SELECT AVG(INDUS) AS AVG_INDUS1 FROM boston
      WHERE CHAS = 1
      ),
      AVG_NOX_CHAS_1 AS(
        SELECT AVG(NOX) AS AVG_NOX1 FROM boston
      WHERE CHAS = 1
      ),
      AVG_NOX_CHAS_0 AS(
        SELECT AVG(NOX) AS AVG_NOX FROM boston
      WHERE CHAS = 0
      )

       SELECT 'CRIM' AS ColumnName
      FROM AVG_CRIM_CHAS_0, AVG_CRIM_CHAS_1 WHERE AVG_CRIM_CHAS_1.AVG_CRIM1 > AVG_CRIM_CHAS_0.AVG_CRIM
      UNION ALL
       SELECT 'ZN' AS ColumnName
      FROM AVG_ZN_CHAS_0, AVG_ZN_CHAS_1 WHERE AVG_ZN_CHAS_1.AVG_ZN1 > AVG_ZN_CHAS_0.AVG_ZN
      UNION ALL
       SELECT 'INDUS' AS ColumnName
      FROM AVG_INDUS_CHAS_0, AVG_INDUS_CHAS_1 WHERE AVG_INDUS_CHAS_1.AVG_INDUS1 > AVG_INDUS_CHAS_0.AVG_INDUS
      UNION ALL
       SELECT 'NOX' AS ColumnName
      FROM AVG_NOX_CHAS_0, AVG_NOX_CHAS_1 WHERE AVG_NOX_CHAS_1.AVG_NOX1 > AVG_NOX_CHAS_0.AVG_NOX

    ''',
    con,
)
#В домах возле реки в среднем меньше промышленных площадей, также в них в среднем выше концентрация оксида азота

,ColumnName
0,INDUS
1,NOX


8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [36]:
tasks+=1
pd.read_sql(
    '''
    WITH Percentiles AS (
  SELECT
    NTILE(10) OVER (ORDER BY INDUS) AS Perc_INDUS,
    NTILE(10) OVER (ORDER BY NOX) AS Perc_NOX,
    INDUS,
    NOX
  FROM boston
)
SELECT
  DISTINCT Perc_INDUS * 10 AS Percentile,
  (SELECT INDUS FROM Percentiles WHERE Perc_INDUS = p.Perc_INDUS LIMIT 1) AS Value_INDUS,
  (SELECT NOX FROM Percentiles WHERE Perc_NOX = p.Perc_NOX LIMIT 1) AS Value_NOX
FROM Percentiles p
ORDER BY Percentile;

    ''',
    con,
)
# Чем выше доля промышленной застройки, тем выше концентрация оксодов азота

,Percentile,Value_INDUS,Value_NOX
0,10,0.46,0.3850
1,10,0.46,0.4280
2,10,0.46,0.5070
3,10,0.46,0.4429
4,10,0.46,0.4720
5,20,2.93,0.3850
6,20,2.93,0.4280
7,20,2.93,0.4429
8,20,2.93,0.4720
9,20,2.93,0.5380


In [39]:
if tasks==8:
    print('Выполнены все задания 🙂')
else:
    print('Выполнены не все задания! 🙀')

Выполнены все задания 🙂
